# StackOverflow Analysis: Milestone 1

In [41]:
### Imports, functions and global variables
import sys
import os
import re
from bs4 import BeautifulSoup as Soup
from collections import Counter
from collections import defaultdict
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
#import pyspark
#import numpy as np
#from pyspark import SparkContext
#from pyspark.mllib.fpm import FPGrowth

#SparkContext.setSystemProperty('spark.executor.memory','3g')
#sc = pyspark.SparkContext('local[*]')

## Global variables

totalquestions=0
totalQuestionswithacceptedanswers=0
totalQuestionswithnoacceptedanswers=0
bodydict=dict()

posts_by_tag = defaultdict()
tags_by_year = defaultdict()
tags_by_post_id = defaultdict()
userdict=dict()
tags=list()

acceptedanswers=list()
bestuser=list()

def populate_posts_by_tag(tag,post_id):
    global posts_by_tag
    if tag == '':
        return
    if post_id == '':
        return

    if tag not in posts_by_tag:
        posts_by_tag[tag] = set()
    posts_by_tag[tag].add(int(post_id))

def populate_tags_by_post_id(id,tag):
    global tags_by_post_id
    if tag == '':
        return
    if id == '':
        return

    if id not in tags_by_post_id:
        tags_by_post_id[id] = list()
    tags_by_post_id[id].append(tag)
    
def populate_tags_by_year(tag,year):
    global tags_by_year
    if tag == '':
        return
    if year == '':
        return

    if tag not in tags_by_year:
        tags_by_year[tag] = list()
    tags_by_year[tag].append(year)

def plot_tags(tagCount):
    
    x,y = zip(*tagCount)

    colormap = plt.cm.gist_ncar #nipy_spectral, Set1,Paired  
    colors = [colormap(i) for i in np.linspace(0, 0.8,50)]   

    area = [i/100 for i in list(y)]   # 0 to 15 point radiuses
    plt.figure(figsize=(8,8))
    plt.ylabel("Number of question associations")
    for i in range(len(y)):
        plt.plot(i+0.5,y[i], marker='o', linestyle='',ms=area[i],label=x[i])

    plt.legend(numpoints=1)
    plt.show()
    
def parse_posts(file):
    global totalquestions,totalQuestionswithacceptedanswers,totalQuestionswithnoacceptedanswers,bodydict

    count=0
    with open(file, encoding='utf8') as handler:
        soup = Soup(handler,"html.parser")
        for message in soup.findAll('row'):
            msg_attrs = dict(message.attrs)
            id = msg_attrs['id']     

            if 'tags' in msg_attrs:
                year =  msg_attrs['creationdate'].split("-")[0]
                #print(year)
                msg_attrs=(msg_attrs[u'tags'].replace('<','').replace('>',','))
                for msg in msg_attrs.split(','):
                    tags.append(msg)
                    populate_posts_by_tag(msg,id)
                    populate_tags_by_year(msg,year)
                    populate_tags_by_post_id(id,msg)
            
            msg_attrs = dict(message.attrs)
            posttypeid = msg_attrs['posttypeid']
            
            if posttypeid == "1":
                totalquestions+=1
                if 'body' in msg_attrs:
                    #id=(msg_attrs[u'id'])
                    body=(msg_attrs[u'body'])
                    bodydict[id]=Soup(body,"html.parser").getText()
                
                if 'acceptedanswerid' in msg_attrs:
                    totalQuestionswithacceptedanswers+=1
                    acceptedanswerid = msg_attrs['acceptedanswerid']
                    acceptedanswers.append(acceptedanswerid)
                else:
                    totalQuestionswithnoacceptedanswers+=1
            else:
                if id in acceptedanswers and 'owneruserid' in msg_attrs:
                     bestuser.append(msg_attrs['owneruserid'])
                    
                #count=count+1

            # if count == 5:
            #     break
            
def parse_users(file):
    global userdict
    count=0
    with open(file, encoding='utf8') as handler:
        soup = Soup(handler,"html.parser")
        for message in soup.findAll('row'):
            msg_attrs = dict(message.attrs)
            id=(msg_attrs[u'id'])
            username=(msg_attrs[u'displayname'])
            userdict[id]=username                   

In [42]:
parse_posts("C:\\Users\\samym\\Downloads\\programmers.stackexchange.com\\Posts2.xml")
tags=list(filter(None, tags))
counts=dict()

top_20_words = Counter(tags).most_common(20)
print("Top 20 most common tags, and the number of their occurences :", top_20_words)

Top 20 most common tags, and the number of their occurences : [('java', 3511), ('c#', 3007), ('design', 2886), ('design-patterns', 2475), ('object-oriented', 2036), ('c++', 1822), ('algorithms', 1732), ('php', 1667), ('architecture', 1639), ('javascript', 1578), ('licensing', 1269), ('database', 1256), ('web-development', 1223), ('programming-languages', 1205), ('python', 1183), ('programming-practices', 1167), ('.net', 1084), ('unit-testing', 1063), ('c', 958), ('testing', 956)]


In [43]:
print(len(bodydict.keys()))

42782


In [44]:
import re
newbodydict=dict()
count=0
from textblob import TextBlob
for k,v in bodydict.items():
    blob = TextBlob(v)
    blob=set(blob.noun_phrases)
    blob=' '.join(blob)
    blob=re.sub('[^A-Za-z0-9 ]+', '', blob)
    newbodydict[k]=blob
    count+=1
    if count == 4000:
        break

print(len(newbodydict.keys()))

4000


In [5]:

count=0

for k,v in bodydict.items():
    if count == 18001:
        print(k,v)
        break
    count+=1

#print(len(newbodydict.keys())) 

188337 I am working on a user driven cms site.  Think of it like stack overflow.  The users can write posts, and publish them to the web.  I am using a WSGI editor to generate the content.
One of the features I am implementing is the ability for users to comment on particular section of a user post. I am trying to think of a clean way to implement this feature.
My current plan is to customize the WYSIWYG editor to generate a unique id per <p> and then store the comments separately, and when the page loads also read the comments and do some jquery magic. All users that are going to be generating content will be admin users, so I am not afraid of someone messing around with ids.
This is one approach I have found, but I was hoping to either get some feedback or alternative ways to accomplish what I need.  Another thing I am trying to figure out is how I can handle a user editing a post that has been commented on... 



In [45]:
import nltk
#nltk.download()
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

# Inspect top 20 tags for this data set.

Each post on the website can be associated with multiple tags, which classify the post into a broad topic/category.
By counting tag occurences, we determine the top 20 tags. This reflects the popularity of programming topics that users have questions about on the website.

In [46]:
totalvocab_stemmed = []
totalvocab_tokenized = []

for k,v in newbodydict.items():
    allwords_stemmed = tokenize_and_stem(v)
    totalvocab_stemmed.extend(allwords_stemmed)
    allwords_tokenized = tokenize_only(v)
    totalvocab_tokenized.extend(allwords_tokenized)

In [47]:
import pandas as pd

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.4, max_features=200000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(newbodydict.values())

print(tfidf_matrix.shape)

Wall time: 2.7 s
(4000, 338)


In [49]:
train_matrix=tfidf_matrix[:2700]
print(train_matrix.shape)

(2700, 338)


In [50]:
test_matrix=tfidf_matrix[2700:]
print(test_matrix.shape)

(1300, 338)


In [51]:
terms = tfidf_vectorizer.get_feature_names()
print (terms)

['abstract', 'access', 'activ', 'actual', 'addit', 'agil', 'algorithm', 'altern', 'analysi', 'andor', 'android', 'answer', 'api', 'app', 'applic', 'approach', 'architectur', 'area', 'aspnet', 'background', 'bad', 'base', 'basic', 'benefit', 'big', 'bit', 'book', 'browser', 'bug', 'build', 'busi', 'c', 'ca', 'ca nt', 'case', 'certain', 'chang', 'choic', 'class', 'clear', 'client', 'code', 'code base', 'code review', 'comment', 'commerci', 'common', 'communic', 'communiti', 'compani', 'compil', 'complet', 'complex', 'comput', 'comput scienc', 'concept', 'content', 'control', 'core', 'cost', 'cs', 'css', 'current', 'custom', 'data', 'data structur', 'databas', 'day', 'decent', 'decis', 'depend', 'design', 'design pattern', 'desktop', 'dev', 'develop', 'develop team', 'differ', 'discuss', 'document', 'doe', 'domain', 'duplic', 'easi', 'edit', 'effect', 'email', 'end', 'engin', 'entir', 'environ', 'error', 'exampl', 'experi', 'extern', 'extra', 'featur', 'feel', 'field', 'file', 'final', 'f

In [91]:

from sklearn.cluster import KMeans

num_clusters = 10
km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100, n_init=1,
                verbose=0)

%time km.fit(train_matrix)

clusters = km.labels_.tolist()

Wall time: 394 ms


In [92]:
from sklearn import metrics
labels = km.labels_
metrics.silhouette_score(train_matrix, labels, metric='euclidean')

0.028606826335576402

In [93]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(train_matrix)

In [70]:
def runkmeans(n_clusters_end):
    i=2
    exp_k = i
    exp_sc = -1
    while i <= n_clusters_end:
        print(i)
        km1 = KMeans(n_clusters=i, init='k-means++', max_iter=100, n_init=1,
                    verbose=0)

        km1.fit(train_matrix) 
        labels = km1.labels_
        score=metrics.silhouette_score(train_matrix, labels, metric='euclidean')
        print(score)
        if score > exp_sc:
            exp_sc = score
            exp_k = i
        i+=1
    print(exp_k, exp_sc)

In [ ]:
runkmeans(100)

In [94]:
postlist=list()
for k in newbodydict.keys():
    postlist.append(k)

#print(postlist)

In [95]:
import pandas as pd

Assignments = { 'post': postlist[:2700], 'cluster': clusters }

frame = pd.DataFrame(Assignments, index = [clusters] , columns = ['post', 'cluster'])

In [96]:
frame['cluster'].value_counts()


0    1173
4     221
5     201
7     200
3     197
2     193
8     153
6     153
9     114
1      95
Name: cluster, dtype: int64

In [97]:
clusterdict=dict()
from __future__ import print_function
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
#terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i]:
        print(' %s' % terms[ind], end='')
    print()
    print()
    
    print("Cluster %d Posts:" % i, end='')
    if type(frame.ix[i]['post']) != str:
        for post in frame.ix[i]['post'].values.tolist():
            print(' %s,' % post, end='')
            if i in clusterdict:
                clusterdict[i].append(post)
            else:
                clusterdict[i]=list(post)
            
    else:
        print(frame.ix[i]['post'])
    print() #add whitespace
    print() #add whitespace

Cluster 0: s good languag programm differ time program java user data new design doe use method function window question class thing person specif thank work databas job technic python file real practic common edit problem team current sql manag googl php experi object exampl idea basic nt comput framework compani variabl small bug type bad javascript need book peopl simpl level pattern model ide busi product point great long right big general multipl support number updat m reason particular agil inform control previous error standard code learn string logic site key access structur version tool static microsoft api public scienc complex case line task softwar design pattern stack valu abstract high solut world concept int return free server end net larg gui onlin implement email librari html various project applic engin requir issu read websit major resourc domain interview document app visual process core certain text input architectur tabl obvious skill ll ui old duplic chang xml ye

In [98]:
predictedop=km.predict(test_matrix).tolist()
print(predictedop)

[0, 0, 0, 2, 2, 0, 2, 0, 6, 6, 0, 7, 9, 0, 2, 2, 0, 8, 4, 1, 0, 9, 0, 7, 3, 0, 0, 0, 2, 4, 0, 7, 9, 8, 8, 5, 7, 0, 7, 0, 0, 8, 0, 6, 4, 4, 0, 0, 0, 0, 6, 0, 0, 5, 1, 0, 0, 8, 4, 6, 0, 0, 5, 0, 7, 7, 8, 4, 0, 3, 9, 0, 0, 0, 0, 0, 3, 7, 0, 1, 0, 0, 0, 9, 2, 0, 0, 8, 4, 0, 0, 1, 8, 5, 0, 0, 3, 2, 0, 7, 5, 1, 4, 0, 0, 0, 7, 0, 6, 0, 0, 0, 2, 0, 0, 0, 7, 0, 6, 5, 5, 0, 0, 3, 8, 5, 0, 0, 7, 9, 0, 4, 5, 0, 0, 0, 0, 6, 0, 0, 6, 0, 4, 0, 3, 0, 0, 9, 6, 2, 6, 3, 8, 0, 5, 0, 3, 7, 0, 7, 0, 0, 6, 0, 0, 0, 0, 7, 7, 0, 5, 0, 2, 0, 0, 0, 0, 7, 4, 5, 7, 8, 5, 2, 0, 2, 2, 5, 0, 0, 0, 7, 0, 8, 4, 4, 4, 0, 3, 7, 9, 5, 0, 0, 9, 3, 2, 4, 0, 8, 0, 7, 0, 0, 6, 0, 8, 2, 5, 5, 3, 0, 0, 6, 7, 4, 5, 0, 0, 0, 6, 0, 2, 7, 8, 7, 8, 2, 0, 3, 0, 5, 5, 9, 3, 5, 7, 0, 5, 3, 0, 3, 0, 0, 0, 8, 5, 0, 7, 0, 7, 6, 9, 9, 0, 6, 4, 0, 8, 5, 5, 5, 5, 0, 0, 1, 9, 6, 5, 0, 0, 0, 5, 0, 5, 0, 4, 5, 0, 0, 4, 0, 0, 0, 3, 0, 6, 2, 0, 6, 5, 0, 9, 5, 8, 6, 4, 0, 0, 9, 6, 0, 0, 0, 0, 5, 8, 0, 0, 3, 0, 0, 5, 0, 2, 9, 4, 9, 0, 6, 3, 8, 0, 

In [99]:
X = tfidf_matrix.todense()

In [100]:
from sklearn.decomposition import PCA
reduced_data = PCA(n_components=2).fit_transform(X)


In [101]:
print(reduced_data.shape)

(4000, 2)


In [102]:
labels = km.fit_predict(tfidf_matrix)

In [125]:
labels_color_map = {
    0: '#20b2aa', 1: '#ff7373', 2: '#ffe4e1', 3: '#005073', 4: '#4d0404',
    5: '#ccc0ba', 6: '#4700f9', 7: '#f6f900', 8: '#00f91d', 9: '#da8c49'}

labels_name_map = {
    0: 'cluster 0', 1: 'cluster 1', 2: 'cluster 2', 3: 'cluster 3', 4: 'cluster 4',
    5: 'cluster 5', 6: 'cluster 6', 7: 'cluster 7', 8: 'cluster 8', 9: 'cluster 9'}
marker_map={
    0: 'x', 1: 'o', 2: 'o', 3: 'o', 4: 'o',
    5: 'o', 6:'o', 7: 'o', 8: 'o', 9: 'o'}
import matplotlib.pyplot as plt

In [126]:
fig, ax = plt.subplots()
namelist=list()
scatterlist=list()
for index, instance in enumerate(reduced_data):
    #print(instance)
    # print instance, index, labels[index]
    pca_comp_1, pca_comp_2 = reduced_data[index]
    color = labels_color_map[labels[index]]
    m = marker_map[labels[index]]
    namelist.append(labels_name_map[labels[index]])
    scatterlist.append(ax.scatter(pca_comp_1, pca_comp_2, c=color,marker=m))
    ax.legend(scatterlist,
           namelist,
           scatterpoints=1,
           loc='best')
plt.show()

KeyboardInterrupt: 